In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Narsaq,GL,2021-01-27 21:24:59,60.9167,-46.0500,37.40,32,75,34.52
1,1,Porbandar,IN,2021-01-27 21:28:50,21.6422,69.6093,61.27,48,0,8.95
2,2,Port Alfred,ZA,2021-01-27 21:28:50,-33.5906,26.8910,64.99,94,64,3.00
3,3,Butaritari,KI,2021-01-27 21:28:50,3.0707,172.7902,80.56,79,95,10.42
4,4,Uruzgan,AF,2021-01-27 21:28:50,32.8333,66.0000,32.85,65,0,2.55


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [4]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [5]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

In [7]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [9]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
3,3,Butaritari,KI,2021-01-27 21:28:50,3.0707,172.7902,80.56,79,95,10.42
8,8,Avarua,CK,2021-01-27 21:28:51,-21.2078,-159.7750,89.60,62,20,16.11
12,12,Puerto Ayora,EC,2021-01-27 21:28:51,-0.7393,-90.3518,86.00,62,20,20.71
16,16,Tual,ID,2021-01-27 21:27:53,-5.6667,132.7500,83.68,77,99,17.20
17,17,Manta,EC,2021-01-27 21:26:23,-0.9500,-80.7333,82.40,74,40,8.05
20,20,Georgetown,MY,2021-01-27 21:24:10,5.4112,100.3354,78.01,100,20,5.77
21,21,Saint-Philippe,RE,2021-01-27 21:27:29,-21.3585,55.7679,78.80,69,0,4.61
30,30,Cockburn Town,TC,2021-01-27 21:24:00,21.4612,-71.1419,80.60,54,40,10.36
31,31,Sinnamary,GF,2021-01-27 21:28:54,5.3833,-52.9500,78.37,78,27,11.63
35,35,Chuy,UY,2021-01-27 21:28:54,-33.6971,-53.4616,76.68,77,3,12.62


In [10]:
preferred_cities_df.count()

City_ID       152
City          152
Country       152
Date          152
Lat           152
Lng           152
Max Temp      152
Humidity      152
Cloudiness    152
Wind Speed    152
dtype: int64

In [11]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Butaritari,KI,80.56,3.0707,172.7902,
8,Avarua,CK,89.60,-21.2078,-159.7750,
12,Puerto Ayora,EC,86.00,-0.7393,-90.3518,
16,Tual,ID,83.68,-5.6667,132.7500,
17,Manta,EC,82.40,-0.9500,-80.7333,
20,Georgetown,MY,78.01,5.4112,100.3354,
21,Saint-Philippe,RE,78.80,-21.3585,55.7679,
30,Cockburn Town,TC,80.60,21.4612,-71.1419,
31,Sinnamary,GF,78.37,5.3833,-52.9500,
35,Chuy,UY,76.68,-33.6971,-53.4616,


In [12]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [19]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [25]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [23]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""